In [63]:
import pandas as pd
from scipy.stats import chi2_contingency

In [3]:
data = pd.read_csv('dating_data.csv')
data.head(7)

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1
5,227,162,0,0
6,435,16,1,0


### Проанализируем датасет

In [4]:
data.shape

(14514, 4)

In [19]:
# проверка на отсутсвие значений
data.isna().sum()

user_id_1    0
user_id_2    0
group        0
is_match     0
dtype: int64

In [20]:
# проверим число уникальных пользователей
data.user_id_1.nunique()

1000

In [21]:
data.user_id_2.nunique()

1000

In [16]:
data.query('user_id_1 == user_id_2')

,user_id_1,user_id_2,group,is_match


In [23]:
data.query('user_id_1 == 79')

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
284,79,664,1,0
2286,79,207,1,0
5448,79,966,1,0
5625,79,165,1,0
5731,79,767,1,0
6618,79,90,1,0
8161,79,6,1,1
9383,79,115,1,0
11237,79,890,1,1


In [28]:
# проверим количество уникальных пар
data_pairs = data.drop(columns = ['group', 'is_match'])
data_pairs.drop_duplicates()

,user_id_1,user_id_2
0,79,91
1,716,353
2,423,677
3,658,165
4,969,155
...,...,...
14509,476,631
14510,455,770
14511,260,6
14512,885,812


In [33]:
# проверим количество наблюдений в каждой из групп (group)
data.query('group == 0').group.count()

4792

In [35]:
data.query('group == 1').group.count()

9722

In [37]:
data.query('group == 0').group.count() + data.query('group == 1').group.count() == data.shape[0]

True

### Вывод по датасету: 
1. Пустые значения отсутствуют 
2. Количество уникальных пользователей не соответствует числу пользователей --> Каждый пользователь каждой из "групп идентификаторов" может встретиться не один раз
3. Каждая пара является уникальной, т.к. количество пар соответствует размерности датасета
4. Сумма количества наблюдений в каждой из групп соответствует размерности датасета (Посчитано количество наблюдений в каждой группе)

### Как оценить качество данного сервиса?
Для этого определим целевую метрику. Т.к. новый алгоритм должен искать наиболее подходящую пару, то количество лайков/мэтчей должно вырасти (причем, скорее всего, значительно). Следовательно, целевой метрикой будет являться количество мэтчей.

In [50]:
g00 = data.query('group == 0 and is_match == 0').is_match.count()
g00

3858

In [54]:
g01 = data.query('group == 0 and is_match == 1').is_match.count()
g01

934

In [57]:
g10 = data.query('group == 1 and is_match == 0').is_match.count()
g10

5813

In [56]:
g11 = data.query('group == 1 and is_match == 1').is_match.count()
g11

3909

In [59]:
# проверим, что никакие данные не были потеряны
g00+g01+g10+g11

14514

In [69]:
# построим таблицу сопряженности
df = pd.DataFrame({'no_match':[g00, g10], 'match':[g01, g11]})
df.index.name = 'group'
df

,no_match,match
group,,
0,3858,934
1,5813,3909


### Какие проверяем гипотезы?
* Н0 - Количество (доли) мэтчей в группах равны
* Н1 - Имеется значимое различие в количестве мэтчей в группах 

*Чтобы отвергнуть Н0 необходимо иметь уровень значимости меньше 0.05*

In [71]:
# применим хи2
result = chi2_contingency(df)
print(f'statistic: {result[0]}')
print(f'p-value: {result[1]}')

statistic: 618.6889141576198
p-value: 1.4418299163662586e-136


### Вывод:
Уровень значимости сильно ниже установленного порогового значения --> можно утверждать, что различие в долях мэтчей является статзначимым

*Другими словами новый алгоритм делает работу сервиса более качественной*